In [18]:
import sys
import os
from pyemd import emd
import numpy as np
from gensim import models
from gensim.models import word2vec
import numpy.linalg as la
from gensim.models.word2vec import Word2Vec
import scipy.spatial as spt
from scipy.spatial import distance
from pyemd import emd
from sklearn.metrics import euclidean_distances


In [19]:
word_vec = None
array = None
filename1 = 'data/dataWMD/1'
filename2 = 'data/dataWMD/2'
filename3 = 'data/dataWMD/3'
removed_words = ['sdkls']


In [20]:
W = np.memmap("data/embed.dat", dtype=np.double, mode="r", shape=(3000000, 300))
with open("data/embed.vocab") as f:
    vocab_list = map(str.strip, f.readlines())


In [21]:
vocab_dict = {w: k for k, w in enumerate(vocab_list)}

In [22]:
def normalize(d, target=1.0):
    raw = sum(d.values())
    factor = target/raw
    return {key:value* factor for key,value in d.items()}


In [23]:
with open(filename1) as f:
    words_1 = f.read().split()
    woduplicates = set(words_1)
with open(filename2) as f:
    words_2 = f.read().split()
    woduplicates = woduplicates.union(words_2)

with open(filename3) as f:
    words_3 = f.read().split()
    woduplicates = woduplicates.union(words_3)


In [24]:
#make a matrix of vectors of all the words
def vectors_words(woduplicates):
    word_vec = None
    for item in woduplicates:
        try:
            vec = W[[vocab_dict[item]]]
            vec = np.array(vec)
            if word_vec is None:
                word_vec = vec
            else:
                word_vec = np.vstack((word_vec, vec))
        except KeyError as e:
            global removed_words
            removed_words += [item]
            pass
    return word_vec

In [25]:
def compute_squared_EDM_method5(X):
    D_ = euclidean_distances(X)
    return D_


In [26]:
def dict_array(d):
    global array
    array = None
    for key,value in d.items():
        value = np.array(value)
        if array is None:
            array = value
        else:
            array = np.vstack((array, value))
    return array

In [27]:

word_vec = vectors_words(woduplicates)


In [28]:
vectors_words(woduplicates)
remove = set(removed_words)
woduplicates = woduplicates-remove

dict_1 = dict.fromkeys(woduplicates, 0)
dict_2 = dict.fromkeys(woduplicates, 0)
dict_3 = dict.fromkeys(woduplicates, 0)  


In [29]:

file1=open(filename1,"r+")
for word in file1.read().split():
    if word in dict_1:
        dict_1[word] += 1
print(dict_1)
dict_1 = normalize(dict_1)
file2=open(filename2,"r+")
for word in file2.read().split():
    if word in dict_2:
        dict_2[word] += 1
print(dict_2)
dict_2 = normalize(dict_2)
print("<<<>>>>>>")
file3=open(filename3,"r+")
for word in file3.read().split():
    if word in dict_3:
        dict_3[word] += 1
dict_3 = normalize(dict_3)

{'gave': 0, 'concert': 0, 'greets': 0, 'Illinois': 1, 'Obama': 1, 'speaks': 1, 'media': 1, 'Japan': 0, 'press': 0, 'President': 0, 'Chicago': 0, 'band': 0}
{'gave': 0, 'concert': 0, 'greets': 1, 'Illinois': 0, 'Obama': 0, 'speaks': 0, 'media': 0, 'Japan': 0, 'press': 1, 'President': 1, 'Chicago': 1, 'band': 0}
<<<>>>>>>


In [30]:
distance_matrix = compute_squared_EDM_method5(word_vec).astype(np.double)

(12, 12)


In [31]:
d1 = np.squeeze(dict_array(dict_1)).astype(np.double)
d2 = np.squeeze(dict_array(dict_2)).astype(np.double)
d3 = np.squeeze(dict_array(dict_3)).astype(np.double)

In [32]:
print("distance between d1 and d2")
print (emd(d2,d1,distance_matrix))
print("distance between d1 and d3")
print (emd(d1,d3,distance_matrix))


distance between d1 and d2
0.9857075171118326
distance between d1 and d3
1.3404909716651137
